# Generative AI Kitchen Assistant

## 🧠 Problem: What should I cook with what I have?

It's a common daily dilemma — staring at your pantry or fridge, trying to decide what to make. Traditional recipe websites need structured input or specific dish names, but real people often think the following when trying to look for what to make next:

- "I want to bake something sweet"
- "I have tomatoes, spinach, and eggs — what can I cook?"
- "Give me a quick vegan dinner"

The problem: natural language is messy. Recipes are messy too. We need something smarter.

#### Home cooks often struggle to find recipes that align with their available ingredients, dietary restrictions, and culinary preferences. This project aims to simplify meal planning by leveraging generative AI to suggest personalized recipes based on user inputs.

### 🤖 Enter Generative AI

This notebook shows how GenAI can bridge the gap between how people talk and structured data like recipes — extracting preferences from natural language and recommending dishes intelligently, even when data is incomplete or vague.


### This notebook is part of the submission for the Kaggle & Google 5-day Generative AI course

My main idea for this project is to have a chat where essentially the AI functions to suggest recipes based on the ingredients a user has and/or what kind of food they prefer to eat. 

The main areas of Generative AI that I learned during the 5-day course and will be included in this project are
1. Structured output/JSON mode/controlled generation
2. Few-shot prompting
3. Retrieval augmented generation (RAG)
4. Vector search/vector store/vector database
5. Grounding
6. Embeddings

---
## Download and Import packages (google-genai & chromadb)
First step is to download and import the required packages
- genai for the Gemini Model
- Markdown used for displaying output from the model
- pandas to read in the dataset of recipes
- json for the structured output
- chromadb to store embeddings of the recipes

In [ ]:
!pip uninstall -qqy jupyterlab jupyterlab-lsp # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0" "chromadb==0.6.3"

In [ ]:
# ----------------------------
# 🛠️ Imports and Setup
# ----------------------------
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

In [ ]:
# ----------------------------
# 🛠️ Imports and Setup
# ----------------------------
import pandas as pd
import ast
import json
import re
import typing_extensions as typing
from typing import Dict, List, Optional

## Set up API key
To run the code and access the Gemini models, an API key is required and is accessible at [AI Studio](https://aistudio.google.com/apikey). Add the API key to `Secrets` under `Add-ons` and name it `"GOOGLE_API_KEY"`.

In [ ]:
# ----------------------------
# 🛠️ API Setup
# ----------------------------
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

## Creating the embedding database with ChromaDB

Create a [custom function](https://docs.trychroma.com/guides/embeddings#custom-embedding-functions) to generate embeddings with the Gemini API. This section of code is based on the codelabs provided by Google & Kaggle.

In the code section, `retrieval_document` generates the document embeddings (the recipes dataset in this case) and `retrieval_query` is used for the query embeddings (a user's request for a recipe).

In [ ]:
# ----------------------------
# 🛠️ Embedding & ChromaDB Setup
# ----------------------------
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

from google.api_core import retry

class GeminiEmbeddingFunction(EmbeddingFunction):
    def __init__(self, document_mode=True):
        self.document_mode = document_mode

    @retry.Retry(predicate=lambda e: isinstance(e, genai.errors.APIError) and e.code in {429, 503})
    def __call__(self, input: Documents) -> Embeddings:
        task_type = "retrieval_document" if self.document_mode else "retrieval_query"
        response = client.models.embed_content(
            model="models/text-embedding-004", 
            contents=input,
            config=types.EmbedContentConfig(task_type=task_type),
        )
        return [e.values for e in response.embeddings]



## Load in the dataset
The dataset of recipes is taken from [RecipeNLG](https://huggingface.co/datasets/mbien/recipe_nlg): A Cooking Recipes Dataset for Semi-Structured Text Generation.

For the preprocessing of the dataset, I have decided to remove entries where the recipe name, ingredients, and instructions were `NULL`. Then these values are combined into one column to create a "document" for each recipe. 

In [ ]:
# ----------------------------
# 📘 Load and Filter Recipes
# ----------------------------
df = pd.read_csv('/kaggle/input/recipes/full_dataset.csv')
df = df.dropna(subset=['title', 'ingredients', 'directions','NER'])
df['full_text'] = df['title'] + ' | ' + df['ingredients'] + ' | ' + df['directions']

## Defining the functions

Each function is written to work with the proposed workflow of how I would like the chat to flow and/or behave.

---
### Extracting user preferences
This function works to take in the user's input and then calls on the Gemini model to extract the user's preferences from the input into a JSON format. The prompt that is fed to the model also includes few-shot prompting with a few provided examples on how the model's response should look. 

### AI use:
With this example of using few-shot prompting and and obtaining a structured output, we're making use of the key NLP characteristics as it interprets user inputs to extract relevant information such as available ingredients, dietary preferences, and desired cuisine types.

In [ ]:
# ----------------------------
# 🔍 Few-Shot Prompt to Extract Preferences
# ----------------------------

# Create a schema for the ingredients and preferences a person has
# Preferences can include diet, cuisine, dislikes, etc.

class Preferences(typing.TypedDict, total=False):
    ingredients: list[str]
    diet: str
    cuisine: str
    dislikes: list[str]
    meal_type: str
    difficulty: str
    dish: str


def extract_preferences(user_input: str) -> Dict:
    """Function to take in the user's input of preferences and then extract them into a JSON format"""
    
    prompt = f"""
    You are a kitchen assistant. Extract structured cooking preferences from user input and return them as JSON.
    
    Return these keys:
    - ingredients (list of strings)
    - cuisine (string)
    - diet (string)
    - dislikes (list of strings)
    - difficulty (string)
    - dish (string – if a specific dish is mentioned by the user otherwise keep it empty)
    
    If the user says they "don't have", "don't want", or "without" an ingredient, include it in dislikes.
    And if the user mentions a dietary preference such as "vegetarian" and "pescatarian", make sure that is stored under
    "diet" and not "dish".
    
    Use empty strings/lists if something isn’t mentioned.
    
    EXAMPLE 1:
    Input: "I want to make lasagna"
    Output: {{
      "ingredients": [],
      "cuisine": "",
      "diet": "",
      "dislikes": [],
      "difficulty": "",
      "dish": "lasagna"
    }}

    EXAMPLE 2:
    Input: "Make me something vegetarian without olives. Something Italian maybe?"
    Output: {{
      "ingredients": [],
      "cuisine": "Italian",
      "diet": "vegetarian",
      "dislikes": ["olives"],
      "difficulty": "",
      "dish": ""
    }}

    EXAMPLE 3:
    Input: "I have tomatoes, pasta, and tuna, but I don't have cheese"
    Output: {{
      "ingredients": ["tomatoes", "pasta", "tuna"],
      "cuisine": "",
      "diet": "",
      "dislikes": ["cheese"],
      "difficulty": "",
      "dish": ""
    }}
    
    USER INPUT:
    Input: "{user_input}"
    Output:
    """
    response = client.models.generate_content(
        model="gemini-2.0-flash",                    # Change the model if JSON not parsed
        config=types.GenerateContentConfig(
            temperature=0.1,
            response_mime_type="application/json",
            max_output_tokens=512,
            response_schema=Preferences
        ),
        contents=[prompt]
    )
    # print(response.text)
    return json.loads(response.text.strip())


### Creating DB of embeddings

In [ ]:
# ----------------------------
# 🛠️ Embedding & ChromaDB Setup
# ----------------------------
chroma_client = chromadb.Client()

def create_recipe_collection(name="recipes", embedding_fn=None):
    """Creates the chromadb named 'recipes' and applies the embedding function, returns the DB of embeddings"""
    existing_collections = chroma_client.list_collections()
    if name in existing_collections:
        chroma_client.delete_collection(name)
    return chroma_client.create_collection(name=name, embedding_function=embedding_fn)

# Initialize embedding function (in document mode for DB creation)
embed_fn = GeminiEmbeddingFunction(document_mode=True)
collection = create_recipe_collection(name="recipes", embedding_fn=embed_fn)



### Filtering recipes based on preferences
I decided to filter the recipes based on the user's preferences before creating embeddings for those recipes to keep API requests limited, considering the fact that the the recipes dataset is large (~2.2 GB)

In [ ]:
# ----------------------------
# ✅ Recipe Filtering & Ingestion
# ----------------------------
def filter_and_embed_recipes(prefs: Dict, df: pd.DataFrame) -> Optional[pd.DataFrame]:
    """
    Filter the recipes based on user preferences and apply the embedding function 
    on the filtered dataset. Returns the filtered DataFrame or None if no matches found.
    """
    global collection
    
    filtered = df.copy()

    # Filter by dish name
    if prefs.get("dish"):
        filtered = filtered[filtered['title'].str.contains(prefs["dish"], case=False, na=False)]

    # Filter by included ingredients
    for ing in prefs.get("ingredients", []):
        filtered = filtered[filtered['NER'].str.contains(ing, case=False, na=False)]

    # Exclude disliked ingredients
    for dislike in prefs.get("dislikes", []):
        filtered = filtered[~filtered['NER'].str.contains(dislike, case=False, na=False)]

    # Limit to top 100 results
    filtered = filtered.head(100)

    # Check if anything remains
    if filtered.empty:
        print("⚠️ No matching recipes found in local data.")
        return None

    # Recreate collection with filtered recipes
    embed_fn.document_mode = True  # switch to document embedding
    collection = create_recipe_collection(name="recipes", embedding_fn=embed_fn)

    documents = filtered['full_text'].tolist()
    metadatas = filtered[['title', 'ingredients', 'directions']].to_dict(orient='records')
    ids = [str(i) for i in filtered.index]

    # Only add if all lists are non-empty
    if documents and metadatas and ids:
        collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )
        return filtered
    else:
        print("⚠️ Filtered dataset is empty after processing. Skipping ChromaDB embedding.")
        return None


### Look for a suitable recipe
Query the dataset for suitable recipes based on the user preferences

In [ ]:
# ----------------------------
# ✅ RAG Query Function
# ----------------------------
def find_best_recipe(prefs: Dict) -> Dict:
    """"""
    query_parts = prefs.get("ingredients", [])
    if prefs.get("dish"):
        query_parts.append(prefs["dish"])
    query = " ".join(query_parts)

    embed_fn.document_mode = False  # switch to query embedding
    results = collection.query(query_texts=[query], n_results=1)

    if results['documents']:
        doc = results['metadatas'][0][0]
        doc['full_text'] = results['documents'][0][0]
        return doc
    return None

### Google Search Grounding with Gemini
Resort to using Google Search for recipes where
1. the user wants to Google for a recipe or
2. a recipe isn't available in the dataset

In [ ]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

# ----------------------------
# 🔎 Google Dish Suggestions via Gemini
# ----------------------------

# 🔎 Dish name suggestion from grounded search
def get_dishes_from_google(prefs: Dict) -> List[str]:
    """Get options for dishes the user can make based on preferences where ingredients aren't given"""
    prompt = f"""
    Based on the following preferences, suggest 5 popular real-world dish names:
    - Cuisine: {prefs.get('cuisine', 'any')}
    - Diet: {prefs.get('diet', 'any')}
    - Difficulty: {prefs.get('difficulty', 'any')}
    - Dish Type: {prefs.get('dish', 'any')}

    Return only a Python-style list of strings. No explanation.
    Example: ["Chocolate Cake", "Chocolate Chip Cookies", "Double Chocolate Brownies", "Cinnamon Rolls", "Nutella Cookies"]
    """

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config=config_with_search
    )

    try:
        text = response.candidates[0].content.parts[0].text.strip()
        if text.startswith("```") and "python" in text:
            text = text.split("```")[1].replace("python", "").strip()
        dishes = eval(text)
        return dishes if isinstance(dishes, list) else []
    except Exception as e:
        print("❌ Failed to parse dish suggestions:", e)
        return []


# 🌐 Fallback: grounded recipe search
def grounded_google_search(dish: str, prefs: Dict = None) -> str:
    """Google search for a specific recipe"""
    context = f"authentic recipe for {dish}, include ingredients and step-by-step instructions."
    if prefs:
        if prefs.get("cuisine"):
            context += f" Cuisine: {prefs['cuisine']}."
        if prefs.get("diet"):
            context += f" Diet: {prefs['diet']}."
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=context,
        config=config_with_search
    )
    return response.text.strip()



### Compare ingredients
For instances where a user provides which ingredients they have, compare the ingredients to the ones in the suggested recipe and state them in the output.

In [ ]:
# ----------------------------
# 🔧 Ingredient Diff Checker
# ----------------------------
def get_missing_ingredients(recipe_ingredients, user_ingredients):
    """Compare user's ingredients to the recipe and return what's missing."""
    lower_user_ings = [i.lower() for i in user_ingredients]
    recipe_ings = [i.lower() for i in recipe_ingredients]  # no split needed

    return [i for i in recipe_ings if not any(u in i for u in lower_user_ings)]



### Edit user preferences
Add in additional preferences if provided by user.

In [ ]:
def merge_preferences(old: dict, new: dict) -> dict:
    """combine previous and new preferences specified by the user"""
    return {
        "ingredients": list(set(old.get("ingredients", []) + new.get("ingredients", []))),
        "dislikes": list(set(old.get("dislikes", []) + new.get("dislikes", []))),
        "cuisine": new.get("cuisine") or old.get("cuisine"),
        "diet": new.get("diet") or old.get("diet"),
        "difficulty": new.get("difficulty") or old.get("difficulty"),
        "dish": new.get("dish") or old.get("dish"),
    }


### Update output
The following functions are to edit the output and have it look more cohesive.

In [ ]:
def safe_parse_list(val):
    if isinstance(val, list):
        return val
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except:
            return [val]
    return [val]

def render_recipe(recipe, user_ingredients=None):
    title = recipe.get('title', 'Untitled Recipe')
    ingredients = safe_parse_list(recipe.get('ingredients', []))
    directions = safe_parse_list(recipe.get('directions', []))

    print("\n👩‍🍳 🍽️ Here's a recipe you might like:\n")
    print(f"🔸 {title.upper()}\n")

    print("🧾 INGREDIENTS:")
    for item in ingredients:
        if isinstance(item, str):
            print(f"  - {item.strip()}")

    print("\n📖 INSTRUCTIONS:")
    for i, step in enumerate(directions, 1):
        if isinstance(step, str):
            print(f"  {i}. {step.strip()}")

    if user_ingredients and len(user_ingredients) > 0:
        missing = get_missing_ingredients(ingredients, user_ingredients)
        if missing:
            print("\n⚠️ YOU MAY NOT HAVE:")
            for item in missing:
                print(f"  - {item.strip()}")


## Chat between the Kitchen Assistant
Main piece of code where the conversation takes place.

In [ ]:
# ----------------------------
# 🤖 Chat Loop
# ----------------------------

prefs = {
    "ingredients": [],
    "dislikes": [],
    "cuisine": "",
    "diet": "",
    "difficulty": "",
    "dish": ""
}

print("👩‍🍳 Welcome to your AI Kitchen Assistant!")
print("👩‍🍳 You can tell me what you want to cook, your ingredients, any dislikes, or preferred cuisine.")
print("👩‍🍳 For example: 'I want to make something vegetarian without mushrooms' or 'I have chicken and rice, give me something easy'.")

while True:
    user_input = input("👤 You: ")
    if user_input.lower() in ['exit', 'quit', 'stop', 'bye']:
        print("👩‍🍳 👋 Bye! Come back anytime.")
        break

    try:
        new_prefs = extract_preferences(user_input)
        prefs = merge_preferences(prefs, new_prefs)
        print("👩‍🍳 📋 Extracted preferences:", prefs)
    except Exception as e:
        print("👩‍🍳 ❌ Couldn't extract preferences. Try again.")
        continue

    # Case 1: High-level preferences (no ingredients)
    if not prefs.get("ingredients") and (prefs.get("cuisine") or prefs.get("diet") or prefs.get("difficulty")):
        dishes = get_dishes_from_google(prefs)
        if not dishes:
            print("👩‍🍳 🤖 Couldn't suggest dishes. Could you try rephrasing?")
            continue

        print("👩‍🍳 🤖 Here are some dishes you might like:")
        for i, dish in enumerate(dishes, 1):
            print(f"{i}. {dish}")

        selected = input("\n👩‍🍳 Which one would you like to make? ").strip()
        print("\n👩‍🍳 🔍 Searching for the recipe...\n")
        recipe_text = grounded_google_search(selected, prefs)
        display(Markdown(recipe_text))

        user_feedback = input("\n👩‍🍳 Is this the kind of recipe you had in mind? (yes/no/stop): ").strip().lower()
        if user_feedback in ['yes', 'stop']:
            print("👩‍🍳 🎉 Enjoy your meal!" if user_feedback == 'yes' else "👩‍🍳 👋 Okay, see you next time!")
            break
        else:
            print("👩‍🍳 🤔 Okay, let's try again.")
            continue

    # Case 2: Ingredients-based search
    filtered = filter_and_embed_recipes(prefs, df)

    if filtered is None or filtered.empty:
        print("👩‍🍳 😕 Couldn't find a good recipe locally. Searching Google...")
        display(Markdown(grounded_google_search(user_input)))
        break

    recipe = find_best_recipe(prefs)

    if recipe:
        render_recipe(recipe, prefs.get('ingredients'))

        user_feedback = input("\n👩‍🍳 Are you happy with this recipe? (yes/no/google/stop): ").strip().lower()
        if user_feedback == 'yes':
            print("👩‍🍳 🎉 Enjoy your meal!")
            break
        elif user_feedback == 'google':
            print("\n👩‍🍳 🔍 Searching Google...")
            display(Markdown(grounded_google_search(user_input)))
            break
        elif user_feedback == 'stop':
            print("👩‍🍳 👋 Okay, see you next time!")
            break
        else:
            print("👩‍🍳 🤔 Okay, let's try again.")
    else:
        print("👩‍🍳 😕 Couldn't find a suitable recipe. Let's try Google.")
        display(Markdown(grounded_google_search(user_input)))
        break


